# Practical Session 5: Path Tracking

## Redes de Sensores y Sistemas Autónomos 
### Grado en Ingeniería de las Tecnologías de Telecomunicación
### Universidad de Sevilla

David Alejo Teissière

## Objectives

In this practical session, we will learn the basics of path following.

In this particular, we will use the Turtlebot simulator so that it can perform two different tasks in a obstacle-free world (empty_world).

+ We will learn to use the basic utilities related to the handling of reference frames in ROS, which is carried out by the `tf`package.
+ We will implement a subscriber to a Goal, which has type geometry_msgs/PoseStamped and make the robot go to the desired location.
+ We will implement a new action server to follow a Path, which consists on a quasi-continuous sequence of Poses.
+ We will introduce the concept of Action, which is like the client/server paradigm, but for longer actions that can include feedback information, such as path tracking.

## TF Basics. Examples and useful tools

ROS has a builtin package to handle transforms of different reference frames. We will learn here the usual reference frames that are present in a simulation and how to retrieve information from the tf package. 

First start a turtlebot simulation in the Docker as seen in Practica_03. Please, refer to Exercise 1b of Practica_03 to see how to move the Turtlebot with the keyboard.

The simulated robot uses many coordinate frames to operate. All of them are published through the ROS `/tf` topic, and can be used by means of the TF library. Among them, we will be using two main frames:

* `odom`, which is the odometry frame. The origin (0,0,0) is set at the first position of the robot when Gazebo is launched
* `base_footprint`, which is a local frame attached to the robot (that is, it moves with the robot), with the X axis aiming to the direction of motion of the robot.

In [ ]:
(bash Docker) > roslaunch turtlebot3_gazebo turtlebot3_empty_world.launch

Then connect to the docker and launch another bash terminal, in which we will execute the teleoperation tool:

In [ ]:
(Powershell) > docker exec -it rssa bash
(bash Docker) > roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch

### Tf echo tool

The `tf` package provides us with a useful tool that shows in real time the transform between two reference frames. In our case, we are interested in the transform between the odometry frame `odom` and the base frame of our turtlebot `base_footprint`.

To show it you should execute in another terminal: 

In [ ]:
(rssa docker) > rosrun tf tf_echo odom base_footprint

Please move the turtlebot and see how the transform changes as the robot moves while the `tf_echo`tool is executing.

### Rqt_tf_tree tool

The rqt UI package provides us with a tool to represent the whole tf tree of the system. The complete tf tree of a turtlebot is shown in Fig. 1. To execute the `rqt_tf_tree` tool please enter:

In [ ]:
(rssa docker) > rosrun rqt_tf_tree rqt_tf_tree


<figure style="text-align:center">
  <img src="images/frames.png" alt="" width=900>
  <figcaption>Fig. 1: Output of the rqt_tf_tree tool.  </figcaption>
</figure>

### Robot state publisher tool (Gazebo)

You may encounter that the tree has fewer leaves than the one of Fig. 1. To correct it, you should open yet another terminal and execute the robot state publisher tool. It publishes al the reference frames of a simulated robot.

Please, connect to another bash terminal of the docker execute the following command, while the `rqt_tf_tree` is running. Once executed, refresh the `rqt_tf_tree` tool.

In [ ]:
(rssa docker) > rosrun robot_state_publisher robot_state_publisher

### Representation from rviz

Stop the rqt_tf_tree tool and launch rviz.

Inside the rviz tool you can add a `Tf` visualization that represents the reference frames in real time in a 3D display.

To execute rviz please run:

In [ ]:
(rssa docker) > rviz

## Tf Listener basics and goal subscriber

Basically, our goal is to make our beloved Turtlebot go to a destination, which is given in a global coordinate frame, such as the `Odometry` frame, or the `Map` frame.

To this end, we will implement a node that receives (subscribes to) the goal. The basic idea here is to transform the pose in global coordinates (i. e. in the odometry or map coordinate frame) to the local frame (base_link frame). The pose in the local coordinate frame can be considered as the position error, or the position vector from the robot to the destination.

To be able to transform the goal pose to local coordinates, we will need a class that is provided in the `tf` package, the TransformListener class. It implements a method that listens to the available transform in ROS and it provides us with some methods to transform between coordinate frames such as `transformPose` and `transformPoint`. For more information, please refer to the tutorial: http://wiki.ros.org/tf/Tutorials/Writing%20a%20tf%20listener%20%28Python%29


__Exercise 1__

+ Create the `turtlebot_controller`package, which depends on the `tf, rospy, roscpp, geometry_msgs, nav_msgs, actionlib, message_generation, message_runtime` packages.
+ Get the skeleton of the `turtlebot_controller` node and copy it to the `scripts` folder of the new package. It implements the `TurtlebotController` class. 
+ Add a subscriber on this node the `move_base_simple/goal` topic with type `geometry_msgs/PoseStamped`, which is linked to the callbackGoal method.
+ In the callback method, the received goal is copied to a goal attribute of the TurtlebotController class and a message is shown for logging purposes. Hint: it is a good idea to have an attribute acting as a flag to indicate that the goal has been received.
+ In the `control_loop` method, which is periodically called, we will do two different things in this exercise. We will use two reference frames which should be defined in the `base_frame_id` and `global_frame_id` attributes. By default, they should be `base_footprint` and `odom`, respectively.
+ First we will show the position of the robot by getting the transform from the `base_frame_id`  to the `global_frame_id`. To this end, we will use the lookuptransform method of the tf listener class.  
+ Then, we will transform the received goal coordinates from the `global_frame_id` to the `base_frame_id`. We will also log it to the screen.


Note: The `base_frame` attribute should be configurable by means of the ROS parameter server, in a private ROS parameter of the node named: `base_frame_id`.


## Basic goal controller for turtlebot

### Control loop for the orientation:

In this section, we will implement a simple control loop we will make the position error go below a given threshold. 

In a first step, we will make the robot point to the destination. In other words, we would like to reduce the angular error to zero. We can do it by using the angular velocity following the control loop depicted in Fig. 2.


<figure style="text-align:center">
  <img src="images/control_loop_diagram.png" alt="" width=700>
  <figcaption>Fig. 2: Control loop block diagram </figcaption>
</figure>

### Control loop for the velocity

Once we have controlled the angular error, we need to command the linear velocity which will be a function of the time and/or the distance to the destination. Besides, we will use the angular error into consideration. 

#### Velocity as a function of the time and distance 

Usually, the commanded velocity to the turtlebot should follow a trapezoidal profile in which we distinguish three different zones (see Fig. 3):

+ Acceleration zone: in the first stage, we gradually increment the speed of the robot until it reaches its cruise speed.
+ Cruise zone: in this section, the cruise speed is maintained. This maximum speed should be should be stored in an attribute of the class and should be retrieved from the parameter server.
+ Deceleration zone: last, we gradually decrement the speed until the distance is below a given threshold `d_thres`. This parameter should be stored in an attribute of the class and should be retrieved from the parameter server.

<figure style="text-align:center">
  <img src="images/trapezoidal.png" alt="" width=700>
  <figcaption>Fig. 3: Examples of Velocity profiles. (a) Trapezoidal (b) 3rd order S-Curve Profile. Source: M. Akdag, H. Sen: "S-curve Motion Profile Design for Vibration Control of Single Link Flexible Manipulator". DEÜFMD 23(68), 661-676 </figcaption>
</figure>

#### Velocity as a function of the angular error.

The idea is that if the angular error is too large, the robot should reduce its velocity. To this end we can use different multipliers (0.9, 0.7, 0.5...) when the error exceeds some thresholds. 

__Exercise_2__

Implement a basic controller in which, if a new goal is received and we have not reached it makes the Turtlebot go to the desired destination.

To this end, you should add a function to calculate the angular and linear velocity commands that will be sent to the robot in the `control` method of the `TurtlebotController` class.


## Path Tracking exercise

When navigating in complex environments, it is a good idea to provide the robot not with a mere goal destination, but also with a safe path to reach it. To this end, we can use the `Path` message, which is defined in the `nav_msgs` package. 

There are plenty of methods that can be used for path following. As a first approximation, the students are encouraged to calculate the closest point from the robot to the path (see point C in Fig. 3). Then, the robot should be commanded to reach not that point but another point farther away (G). The distance between C and G is usually called lookahead (L). At each new iteration in our control module, the point to be reached is to be calculated again, and then we use the same method that we implemented in Exercise 2. 

<figure style="text-align:center">
  <img src="images/path_tracking.png" alt="" width=400>
  <figcaption>Fig. 3: Path tracking. C is the closest point from the Path to the robot. L is the lookahead distance. G is the current goal point.</figcaption>
</figure>

__Exercise_3__

Add a Path subscriber in your `turtlebot_controller` node and then implement a Callback function to the `path` topic of type `nav_msgs/Path`. You should reflect in the class the current path that is being tracked and that a path is being executed.

Implement the path tracking algorithm, which in each iteration obtains the current goal point by the strategy proposed in this section, or a similar one. Note that all the parameters of your path tracker, such as the lookahead distance, should be configurable by means of the ROS parameter server.

To test the path subscriber and follower that you have developed, we provide you with a the `path_publisher.py` tool. You can find the code of the tool in the `rssa` repository of the class, in the `practica_5` folder. This tool publishes a path in a given reference frame, which is loaded with the parameter server by using a convenient launch file `launch/test_path_publisher.launch`. You can select between two test paths, which are on the `test` folder. Please attend to the explanation of the professor for more details.

## Actionlib package. Managing complex actions in ROS.

In any large ROS based system, there are cases when someone would like to send a request to a node to perform some task, and also receive a reply to the request. This can currently be achieved via ROS services.

In some cases, however, if the service takes a long time to execute, the user might want the ability to cancel the request during execution or get periodic feedback about how the request is progressing. The actionlib package provides tools to create servers that execute long-running goals that can be preempted. It also provides a client interface in order to send requests to the server. 

An action is composed of:

+ A goal to be reached, in our case a path. 
+ The result of the action, which in our case can be a mere boolean indicating whether the goal has been successfully reached or not. We might want to include a string so that the causes of error are given to the user.
+ Finally, there is a feedback message which the server should periodically send to the client. It indicates the current progress of the task. In our case, we can provide the user with the id number of the current goal that is being tracked and the distance from the robot to the current goal. 

This gives us the following action file, which should be included in the `action` folder of your package:

__Exercise_4__

* Follow the ROS actionlib tutorials (http://wiki.ros.org/actionlib/Tutorials) and implement a basic Action Server that provides the Path tracking service. 

* Implement a basic Path Action client to test it, based on the provided `path_publisher.py` tool.



## Summary

In this practical session, you should have learned the following:

+ How to transform points from global to local coordinate frames.
+ Implement a basic control law that allows the Turtlebot to reach a destination.
+ Extend the control law to allow the Turtlebot to follow continuous paths.
+ The concept of lookahead distance.
+ Implement and use action servers.